In [1]:
%%configure -f
{
    "conf" : { 
        "spark.executor.instances" : "4" ,
        "spark.executor.cores" : "1" ,
        "spark.executor.memory" : "2g" ,
        "spark.driver.memory" : "4g"
    }
}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
655,application_1765289937462_0648,pyspark,idle,Link,Link,None,
656,application_1765289937462_0649,pyspark,idle,Link,Link,None,
657,application_1765289937462_0650,pyspark,idle,Link,Link,None,
664,application_1765289937462_0657,pyspark,idle,Link,Link,None,
665,application_1765289937462_0658,pyspark,idle,Link,Link,None,
666,application_1765289937462_0659,pyspark,idle,Link,Link,None,
667,application_1765289937462_0660,pyspark,idle,Link,Link,None,
668,application_1765289937462_0661,pyspark,idle,Link,Link,None,
669,application_1765289937462_0662,pyspark,busy,Link,Link,None,
670,application_1765289937462_0663,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, lower, udf
from pyspark.sql.types import StringType
import time

# Paths για τα crime data
crime_2010_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
crime_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"

# Schema – ίδιο όπως πριν
crime_schema = StructType([
    StructField("DR_NO",          StringType()),
    StructField("Date Rptd",      StringType()),
    StructField("DATE OCC",       StringType()),
    StructField("TIME OCC",       StringType()),
    StructField("AREA",           StringType()),
    StructField("AREA NAME",      StringType()),
    StructField("Rpt Dist No",    StringType()),
    StructField("Part 1-2",       StringType()),
    StructField("Crm Cd",         StringType()),
    StructField("Crm Cd Desc",    StringType()),
    StructField("Mocodes",        StringType()),
    StructField("Vict Age",       IntegerType()),
    StructField("Vict Sex",       StringType()),
    StructField("Vict Descent",   StringType()),
    StructField("Premis Cd",      StringType()),
    StructField("Premis Desc",    StringType()),
    StructField("Weapon Used Cd", StringType()),
    StructField("Weapon Desc",    StringType()),
    StructField("Status",         StringType()),
    StructField("Status Desc",    StringType()),
    StructField("Crm Cd 1",       StringType()),
    StructField("Crm Cd 2",       StringType()),
    StructField("Crm Cd 3",       StringType()),
    StructField("Crm Cd 4",       StringType()),
    StructField("LOCATION",       StringType()),
    StructField("Cross Street",   StringType()),
    StructField("LAT",            DoubleType()),
    StructField("LON",            DoubleType())
])

# Implementation 2: DataFrame API (με UDF)

spark = SparkSession \
    .builder \
    .appName("DF Query 1 execution (with UDF)") \
    .getOrCreate()

print("====================================================================")
print("Reloading crime data as DataFrames with explicit schema...")

t0 = time.time()

crime_2010_df = spark.read.format("csv") \
    .options(header="true") \
    .schema(crime_schema) \
    .load(crime_2010_path)

crime_2020_df = spark.read.format("csv") \
    .options(header="true") \
    .schema(crime_schema) \
    .load(crime_2020_path)

crime_df = crime_2010_df.unionByName(crime_2020_df)

t1 = time.time()
print(f"[INFO] Data loading + union: {t1 - t0:.2f} sec")

# -----------------------------------------
# UDF, όπως calculate_yearly_income_udf
# -----------------------------------------
def categorize_victim_age(age):
    """
    Επιστρέφει ηλικιακή ομάδα με βάση την ηλικία.
    Αν age είναι None, <=0 ή μη αριθμός → None.
    """
    if age is None:
        return None
    try:
        a = int(age)
    except Exception:
        return None

    if a <= 0:
        return None
    if a < 18:
        return "Children (<18)"
    elif a <= 24:
        return "Young adults (18-24)"
    elif a <= 64:
        return "Adults (25-64)"
    else:
        return "Seniors (>64)"

categorize_victim_age_udf = udf(categorize_victim_age, StringType())

print("====================================================================")
print("Filtering aggravated assaults and computing age groups (with UDF)...")

t2 = time.time()

aggravated_df = crime_df.filter(
    col("Crm Cd Desc").isNotNull() &
    lower(col("Crm Cd Desc")).contains("aggravated assault")
)

age_grouped_udf_df = aggravated_df \
    .withColumn("age_group", categorize_victim_age_udf(col("Vict Age"))) \
    .filter(col("age_group").isNotNull()) \
    .groupBy("age_group") \
    .count() \
    .orderBy(col("count").desc())

t3 = time.time()

age_grouped_udf_df.show(truncate=False)
print(f"[TIMING] DF (with UDF) pipeline: {t3 - t2:.2f} sec")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
671,application_1765289937462_0664,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reloading crime data as DataFrames with explicit schema...
[INFO] Data loading + union: 3.36 sec
Filtering aggravated assaults and computing age groups (with UDF)...
+--------------------+------+
|age_group           |count |
+--------------------+------+
|Adults (25-64)      |121660|
|Young adults (18-24)|33758 |
|Children (<18)      |10904 |
|Seniors (>64)       |6011  |
+--------------------+------+

[TIMING] DF (with UDF) pipeline: 0.15 sec